# DEEPX Tutorial 03 - AI Porject Workflow with DEEPX NPU

This third tutorial demonstrates the full end-to-end workflow for AI model deployment on DEEPX hardware.

We will train a forklift and worker detection model, utilize the DX-COM tool for conversion to the DXNN format, and execute the final AI application on a DEEPX NPU. This process will provide a comprehensive picture of the DEEPX NPU development pipeline. 

This tutorial is based on dx-all-suite v2.0.0, released in September 2025.

## Hands-on Project Overview

- **Detection classes**: Forklift, Worker
- **Base AI model**: YOLOv7
- **Dataset**: 1448 images about Forklift & Worker from [kaggle](https://www.kaggle.com/datasets/hakantaskiner/personforklift-dataset/data)
- **Train**: Need NVIDIA GPU with more than 24G GRAM
- **Inference NPU**: `DX-M1`
- **AI application**: Modify and reuse the yolo demo of the DX-APP
- **Expected output**:
  ![img](assets/detection-goal.jpg)

## AI Workflow Overview

This diagram explains the common workflow of an AI project.

We define the goal, collect and label data, and train the model.
DX-Compiler helps make the model faster and lighter (INT8) for DX NPU.
The final step is deploying the model to the DEEPX NPU using DX-App or DX-Stream.

Each step builds toward real-world AI solutions, such as worker and forklift detection.
![img](assets/workflow2.jpg)

## 1. AI Workflow - Model Selection based on the use case

To start an AI project, we need to select an AI model that fits the use case.

In this tutorial, our goal is to detect forklifts and workers.
We will use YOLOv7, a well-known model for object detection.

- Choose YOLOv7 to detect Forklift & Worker
- For more details of YOLOv7: 👉 [link](https://docs.ultralytics.com/models/yolov7/)
- How to use YOLOv7: 👉 [link](https://github.com/WongKinYiu/yolov7)

## 2. AI Workflow - Data Preparation & Annotation

Download the forklift-person labeled dataset from Kaggle:
 - Reference: [kaggle link](https://www.kaggle.com/datasets/hakantaskiner/personforklift-dataset)

## 3. AI Workflow - Train

To train the model efficiently, you should use a GPU that has 24GB or more graphic memory.

 - How to train YOLOv7: 👉 [Link](https://colab.research.google.com/drive/1dAdjJuhXqFM_Qcd0QqAn7_AGx7abA5aX?usp=sharing)

## 4. AI Workflow - Optimization with DX-Compiler

Let's try to compile your pre-trained AI model to DXNN format.

Overall processes are:
1. Get a pre-trained mode on pytorch framework
2. Convert it to ONNX format
3. Compile ONNX to DXNN (for more details of DX-COM, refer to the user guide 👉  [here](https://developer.deepx.ai/?files=MjUxNQ==))

The source structure of DX-COM is organized as follows:
```bash
dx_com
 ├── calibration_dataset   # Dataset used to optimize model accuracy <br>
 ├── dx_com
 │ ├── cv2/                # Third party shared libraries (e.g., OpenCV) <br>
 │ ├── google/             # Third party shared libraries (e.g., protobuf) <br>
 │ ├── numpy/              # Third party shared libraries (e.g., NumPy) <br>
 │ ├── ...                 # Other dependencies <br>
 │ └── dx_com              # Core compiler implementation <br>
 ├── sample
 │ ├── MobilenetV1.json    # Sample configuration file <br>
 │ └── MobilenetV1.onnx    # Sample ONNX model
 └── Makefile              # Build script for compiling the sample model <br>
```

In [ ]:
#!pwd
# Move to dx_com directory
%cd dx-all-suite/dx-compiler/dx_com

In [ ]:
!tree -L 1
!tree sample

!cat 

How to create my custom YOLOv7 model:
- [Link](https://colab.research.google.com/drive/1dAdjJuhXqFM_Qcd0QqAn7_AGx7abA5aX?usp=sharing)
- [DXNN](https://drive.google.com/file/d/1e9PEJeS2ZM16Y-5U2HrG9o5jzdvEaIh0/view?usp=sharing)
- [ONNX](https://drive.google.com/file/d/1ZlRppHtz26X1ID8BUHuFMH6EVfPzDOsh/view?usp=sharing)

In [ ]:
!cp ~/Downloads/yolov7-forklift-person.dxnn ./
!pwd

In [ ]:
%cd /home/orangepi/dx-tutorials/dx-all-suite/dx-runtime/dx_app

In [ ]:
#!cp ~/Downloads/yolov7-forklift-person.dxnn ./
!cp ~/yolov7-forklift-person.dxnn ./

In [ ]:
!ls

In [ ]:
DIFF_TEXT = r"""
diff --git a/demos/demo_utils/yolo_cfg.cpp b/demos/demo_utils/yolo_cfg.cpp
index 7866d70..cd8f49d 100644
--- a/demos/demo_utils/yolo_cfg.cpp
+++ b/demos/demo_utils/yolo_cfg.cpp
@@ -144,14 +144,14 @@ YoloParam yolov7_640 = {
     0.3,
     0.4,
     0,
-    80,
+    2,
     "output",
     {
         createYoloLayerParam("onnx::Reshape_491", 80, 80, 3, { 12.0, 19.0, 40.0 }, { 16.0, 36.0, 28.0 }, { 0 }),
         createYoloLayerParam("onnx::Reshape_525", 40, 40, 3, { 36.0, 76.0, 72.0 }, { 75.0, 55.0, 146.0 }, { 1 }),
         createYoloLayerParam("onnx::Reshape_559", 20, 20, 3, { 142.0, 192.0, 459.0 }, { 110.0, 243.0, 401.0 }, { 2 })
     },
-    { "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat", "trafficlight", "firehydrant", "stopsign", "parkingmeter", "bench", "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sportsball", "kite", "baseballbat", "baseballglove", "skateboard", "surfboard", "tennisracket", "bottle", "wineglass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hotdog", "pizza", "donut", "cake", "chair", "couch", "pottedplant", "bed", "diningtable", "toilet", "tv", "laptop", "mouse", "remote", "keyboard", "cellphone", "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors", "teddybear", "hairdrier", "toothbrush"},
+    { "Forklift", "Worker" },
     PostProcType::OD
 };
 
@@ -241,4 +241,4 @@ YoloParam yolov5s_face_640 = {
     },
     {"face"},
     PostProcType::FACE
-};
\ No newline at end of file
+};
"""

# 파일로 저장
with open("dx_app_update.diff", "w", encoding="utf-8") as f:
    f.write(DIFF_TEXT)

print("Saved: dx_app_update.diff (bytes)", len(DIFF_TEXT))

In [ ]:
!pwd
!ls

In [ ]:
!git apply dx_app_update.diff

In [ ]:
!ls

In [ ]:
!./build.sh

In [ ]:
!./bin/yolo -h

In [ ]:
!cp ~/dx-tutorials/assets/forklift-worker.png ./

In [ ]:
!./bin/yolo -m yolov7-forklift-person.dxnn -i forklift-worker.png -p 4

In [ ]:
from IPython.display import Image, display

# Show the image
display(Image(filename="result.jpg"))